In [ ]:
!pip3 install mlflow==1.28

In [ ]:
!pip3 install boto3

In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
    
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifact
import time

In [ ]:
import mlflow
import os
def set_exp(exp_name):
    """
    This function is used to register an experiment in mlflow tracking service.
    The name of the experiment provided by the user will be set in mlflow.
    """
    try:
        usr = "jovyan"
        os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'
        os.environ["AWS_ACCESS_KEY_ID"] = f"minioadmin"
        os.environ["AWS_SECRET_ACCESS_KEY"] = f"minioadmin"
        os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"https://<worker-node-ip>:31900"
        os.environ['MLFLOW_ARTIFACT_ROOT'] = f"s3://<bucket-name>"
        os.environ['MLFLOW_S3_IGNORE_TLS'] = f"true"
        mlflow.set_tracking_uri("https://mlflow.hpe-ezaf.com")
        mlflow.set_experiment(exp_name)
        mlflow.set_tag('mlflow.user', usr)
    except Exception as e:
        print("Failed to set user.")
        print(e)

In [ ]:
# Set up an experiment with set_exp from ezmllib.mlflow
experiment_name = 'mlflow_demo-01'
set_exp(experiment_name)

In [ ]:
print(os.environ)

In [ ]:
# Wine Quality Sample

homedir = !echo $HOME # get user home directory for data
homedir = homedir[0]
print(homedir)

def train(in_alpha, in_l1_ratio):

    print("function call")
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    #csv_url = (
    #    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    #)
    #try:
    #    data = pd.read_csv(csv_url, sep=";")
    #except Exception as e:
    #    logger.exception(
    #        "Unable to download training & test CSV, check your internet connection. Error: %s", e
    #    )
     
    # Read locally
    data_path = homedir + "/shared/mlflow/wine-quality.csv"

    data = pd.read_csv(data_path)
    

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    
        # Execute ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

        # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

    mlflow.end_run() # Executing this ends one run of the current experiment.
    
    #return model_info

In [ ]:
train(0.5, 0.5)